In [1]:
from functions import *
import resources

running


In [2]:
# 加载器，将机器码程序加载到内存中
def loadProgram(file):
    fil = open(file,'r')
    lineno = 0
    address = 0
    while True:
        line = fil.readline()
        lineno += 1
        if (lineno == 1 or lineno == 2):
            continue
        if(line == '' or line == '\n'):
            break
        try:
            instruc = line
            instruc = instruc.strip('\n')
            instruc = instruc.split(',')[0]
            instruc = instruc.split(';')[0]
            instruc = instruc.strip(',')    #指令为字符串类型,十六进制
            instruc = instruc.strip(';')

            print(instruc, len(instruc))
            
            for i in range(0, len(instruc)//2):
                resources.Imem[address] = instruc[2*i : 2*i+2]
                address += 1
        except:
            print(f'File {file} line {lineno} has error')
            pass
    fil.close

In [3]:
loadProgram("test_pipeline2.coe")
print(resources.Imem)
print(resources.reg)
print(resources.PC)

30F00100000000000000 20
10 2
10 2
10 2
30F10200000000000000 20
10 2
10 2
10 2
00 2
['30', 'F0', '01', '00', '00', '00', '00', '00', '00', '00', '10', '10', '10', '30', 'F1', '02', '00', '00', '00', '00', '00', '00', '00', '10', '10', '10', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00'

In [4]:
def check():
   #print(resources.stat)
   if(resources.stat == 1):
      return 1
   elif(resources.stat == 2):
      print("Error: Halt")
      return 0
   elif(resources.stat == 3):
      print("Error: ilegal address")
      return 0
   elif(resources.stat == 4):
      print("Error: ilegal instruction")
      return 0

instruction_no = 0


In [5]:
import threading

# 下面这些是来自于上一个周期的信号，也就是这一周期各个阶段接受的输入信号
# 用于本周期的计算

# (pre_valP, pre_Cnd, pre_valC, pre_icode, pre_valM)
IF_pre_signals = (0, 0, 0, 0, 0)

# (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC) 
ID_pre_signals =(0, 0, 0, 0, 0, 0)

# (ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA) 
EX_pre_signals = (0, 0, 0, 0, 0, 0, 0, 0)

# (EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA) 
MEM_pre_signals = (0, 0, 0, 0, 0, 0, 0, 0, 0)

# (MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA) 
WB_pre_signals = (0, 0, 0, 0, 0, 0, 0, 0, 0)

# ------------------------------------------------------------------------------------------------
# 下面是本周期计算完成产生的结果信号
# 这样分离是为了避免多线程带来的共享变量临界区

# (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC)
IF_result = (0, 0, 0, 0, 0, 0)

# (ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA)
ID_result =(0, 0, 0, 0, 0, 0, 0, 0)

# (EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA)
EX_result = (0, 0, 0, 0, 0, 0, 0, 0)

# (MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA)
MEM_result = (0, 0, 0, 0, 0, 0, 0, 0, 0)

# 目前的写回阶段没有产生新的信号
# TODO
# WB_result = (0, 0, 0, 0, 0, 0, 0, 0, 0)


# -------------------------------------------------------------------------------------------------
# 五个阶段的线程任务函数
# 使用上一个周期的信号进行计算，产生本周期的输出信号
# 在后面的cycle函数中，一个周期的末尾，各个阶段都已经计算完成后，将上个周期的信号值更新为本周期计算产生的新的信号值
def task_fetch():
   global IF_pre_signals, IF_result
   IF_result = Fetch(IF_pre_signals)
   # TODO
   # 目前未考虑任何分支或者跳转指令
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} fetching")

def task_decode():
   global ID_pre_signals, ID_result
   ID_result = Decode(ID_pre_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} decoding")

def task_execute():
   global EX_pre_signals, EX_result
   EX_result = Execute(EX_pre_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} executing")

def task_memory():
   global MEM_pre_signals, MEM_result
   MEM_result = Memory(MEM_pre_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} memorying")

def task_writeback():
   global WB_pre_signals
   WriteBack(WB_pre_signals)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} writingback")


# 模拟一个周期的执行，同一时刻五个阶段（线程）各自执行自己的指令
def cycle():
   # 本周期即将计算的信号和来自上一周期的计算完成的信号
   global IF_result, ID_result, EX_result, MEM_result
   global IF_pre_signals, ID_pre_signals, EX_pre_signals, MEM_pre_signals, WB_pre_signals
   # 创建5个线程，每个线程完成一个阶段的工作
   fetch = threading.Thread(target=task_fetch)
   decode = threading.Thread(target = task_decode) 
   execute = threading.Thread(target=task_execute)
   memory = threading.Thread(target = task_memory)
   writeback = threading.Thread(target = task_writeback)

   fetch.start()
   decode.start()
   execute.start()
   memory.start()
   writeback.start()

   fetch.join()
   decode.join()
   execute.join()
   memory.join()
   writeback.join()

   #本周期末尾，将来自上个周期的信号更新掉
   # TODO
   (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC) = IF_result
   IF_pre_signals = (IF_valP, 0, IF_valC, IF_icode, 0)
   ID_pre_signals = IF_result
   EX_pre_signals = ID_result
   MEM_pre_signals = EX_result
   WB_pre_signals = MEM_result

   if check() == False:
      return False

   # debug
   print(resources.reg)

   return True

In [6]:
no = 0
while True:
    if not cycle():
        break
    print(f"current cycle:{no}")
    no += 1
    print()

# print(resources.mem)
print(resources.reg)

thread Thread-5 fetching
thread Thread-6 decoding
thread Thread-7 executing
thread Thread-8 memorying
thread Thread-9 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:0

thread Thread-10 fetching
thread Thread-11 decoding
thread Thread-12 executing
thread Thread-13 memorying
thread Thread-14 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:1

thread Thread-15 fetching
thread Thread-16 decoding
thread Thread-17 executing
thread Thread-18 memorying
thread Thread-19 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:2

thread Thread-20 fetching
thread Thread-21 decoding
thread Thread-22 executing
thread Thread-23 memorying
thread Thread-24 writingback
[0, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current cycle:3

thread Thread-25 fetching
thread Thread-26 decoding
thread Thread-27 executing
thread Thread-28 memorying
thread Thread-29 writingback
[1, 0, 0, 0, 500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current